In [ ]:
import sys
import os
path = os.path.abspath("../../VecRepV3") 
sys.path.append(path)
print(path)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Sampler, random_split, Dataset
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

from collections import defaultdict
from line_profiler import profile
from scipy.linalg import orthogonal_procrustes

import src.data_processing.BruteForceEstimator as bfEstimator
import src.data_processing.ImageCalculations as imgcalc
import src.visualization.ImagePlots as imgplt
import src.helpers.ModelUtilities as models
import src.data_processing.Utilities as utils
import src.helpers.FilepathUtils as Futils

from src.visualization import SamplingMethod, BFmethod
from src.data_processing.SampleEstimator import SampleEstimator
from functools import partial
from learnable_polyphase_sampling.learn_poly_sampling.layers import get_logits_model, PolyphaseInvariantDown2D, LPS
from learnable_polyphase_sampling.learn_poly_sampling.layers.polydown import set_pool

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
print(device)

In [ ]:
# ----------------------------------Image Input----------------------------------
IMAGE_TYPES = ["NbinMmax_ones", "Nbin", "triangles", "triangle_mean_subtracted"]

IMAGE_FILTERS = ["unique", "Nmax_ones", "one_island"]

IMAGE_PRODUCT_TYPES = ["ncc", "ncc_scaled"]

EMBEDDING_TYPES = ["pencorr_D"]

dimensions = 32

imageType = "shapes_3_dims_6_3" #6x6 triangle in 12x12 matrix shapes_3_dims_6_3
filters = ["unique"]
imageProductType = "ncc_scaled_-1"
overwrite = {"imgSet": False, "imgProd": False, "embedding": False}
weight = None
embeddingType = f"pencorr_{dimensions}"
k=5
percentage = 0.1

imageSet = utils.generate_filtered_image_set(imageType, filters, Futils.get_image_set_filepath(imageType, filters))

imageSet = np.array(imageSet)

testSize = int(percentage * len(imageSet)) 
trainingSize = len(imageSet) - testSize
testSample, trainingSample = SamplingMethod.generate_random_sample(imageSet, testSize, trainingSize)
sampleName = f"{imageType} {filters} {percentage} sample"

sampleEstimator = SampleEstimator(sampleName=sampleName, embeddingType=embeddingType, imageProductType=imageProductType)

index1 = np.random.randint(len(testSample))
index2 = np.random.randint(len(testSample))

input1=testSample[index1]
input2=testSample[index2]

imgplt.plot_original_images(input1, input2, index1, index2)

# ------------------------- Preprocessing Dataset ------------------------
input_dataset = []
for img in testSample:
    img_tensor = torch.from_numpy(np.array(img, dtype=np.float64))
    img_tensor = img_tensor.unsqueeze(0).unsqueeze(0).to(device).double()
    input_dataset.append(img_tensor)
input_dataset = [tensor.float() for tensor in input_dataset] 

stacked_tensor = torch.stack(input_dataset)
input_dataset = stacked_tensor.cpu().numpy()      
input_dataset = [torch.tensor(data).to(device).float() for data in input_dataset]

In [ ]:
model = models.SimpleCNN(dimensions=dimensions, padding_mode='circular').to(device)
model.load_state_dict(torch.load(os.path.join(os.path.abspath("../../VecRepV3"), "model", 
                                               f'best_model_{imageType}_{dimension}d.pt')))
model.eval()

In [ ]:
#----------------------Metric 1 - Loss Calculation-----------------
NCC_scaled_value = imgcalc.get_NCC_score(input1, input2)
print("\nLoss Calculation")
print("\nScaled NCC: ",NCC_scaled_value)

embedded_vector_image1 = model(input_dataset[index1])
embedded_vector_image2 = model(input_dataset[index2])

dot_product_value = imgcalc.get_dp_score(embedded_vector_image1, embedded_vector_image2)

print("Dot product value of model: ", dot_product_value.item())

train_loss_value = imgcalc.get_loss_value(dot_product_value, NCC_scaled_value) 
print("Loss: ", train_loss_value)

matrixG = sampleEstimator.imageProductMatrix  
matrixA = sampleEstimator.embeddingMatrix 
dot_product_matrix = np.dot(matrixA.T, matrixA)
dot_product_value_Pencorr = dot_product_matrix[index1][index2]
difference = abs(dot_product_value_Pencorr - dot_product_value)

print("\nDot product value of BF Method: ", dot_product_value_Pencorr)

train_loss_value = imgcalc.get_loss_value(torch.tensor(dot_product_value_Pencorr), NCC_scaled_value) 
print("Loss: ", train_loss_value)
print("Difference in values of BF Method and Model Method: ", difference.item())

In [ ]:
#----------------------Metric 2 - KNNIoU-----------------
print("----------------\nBrute Force Method -- KNN-IOU score\n----------------")
vectorb_bf1 = matrixG[index1]
vectorc_bf1 = imgcalc.get_vectorc_brute(index1, matrixA)
imgplt.display_and_plot_results(vectorb_bf1, vectorc_bf1, "Brute Force", index1, k, testSample)

vectorb_bf2 = matrixG[index2]
vectorc_bf2 = imgcalc.get_vectorc_brute(index2, matrixA)
imgplt.display_and_plot_results(vectorb_bf2, vectorc_bf2, "Brute Force", index2, k, testSample)

print("----------------\nModel Method -- KNN-IOU score\n----------------")
vectorb_model1 = matrixG[index1]
vectorc_model1 = imgcalc.get_vectorc_model(index1, model, input_dataset)
imgplt.display_and_plot_results(vectorb_model1, vectorc_model1, "Model", index1, k, testSample)

vectorb_model2 = matrixG[index2]
vectorc_model2 = imgcalc.get_vectorc_model(index2, model, input_dataset)
imgplt.display_and_plot_results(vectorb_model2, vectorc_model2, "Model", index2, k, testSample)

In [ ]:
#--------------------Visualisation across dataset-------------------
#input(array_dataset, tensor_dataset)
kscores, losses, ncc_loss_dict = imgcalc.kscore_loss_evaluation(testSample, input_dataset, model, k)

imgcalc.loss_per_ncc_score(ncc_loss_dict)
imgplt.plot_score_distribution(kscores, "K-Score")
imgplt.plot_score_distribution(losses, "Loss")

In [ ]:
#--------------------Visualisation across dataset across dimensions-------------------
dimensions = [32, 64, 128, 192, 256, 384, 512]

for dim in dimensions:
    print(f"Dimension {dim}")
    embeddingType = f"pencorr_{dim}"
    sampleName = f"{imageType} {filters} {percentage} sample"

    sampleEstimator = SampleEstimator(sampleName=sampleName, embeddingType=embeddingType, imageProductType=imageProductType)
    
    input_dataset = []
    for img in testSample:
        img_tensor = torch.from_numpy(np.array(img, dtype=np.float64))
        img_tensor = img_tensor.unsqueeze(0).unsqueeze(0).to(device).double()
        input_dataset.append(img_tensor)
    input_dataset = [tensor.float() for tensor in input_dataset] 
    stacked_tensor = torch.stack(input_dataset)
    input_dataset = stacked_tensor.cpu().numpy()      
    input_dataset = [torch.tensor(data).to(device).float() for data in input_dataset]
    num = len(input_dataset)
    
    model = models.SimpleCNN(dimensions=dim, padding_mode='circular').to(device)
    model.load_state_dict(torch.load(os.path.join(os.path.abspath("../../VecRepV3"), "model", 
                     f'best_model_{imageType}_{dimension}d.pt'), weights_only=True))

    model.eval()

    kscores, losses, ncc_loss_dict = imgcalc.kscore_loss_evaluation(testSample, input_dataset, model, k)
   
    imgcalc.loss_per_ncc_score(ncc_loss_dict)
    imgplt.plot_score_distribution(kscores, "K-Score")
    imgplt.plot_score_distribution(losses, "Loss")



In [ ]:
# -----------------------------Before transforming output embedding matrices-----------------------------
# ----------- Preprocessing dataset -------------
matrixG = sampleEstimator.imageProductMatrix  
matrixA = sampleEstimator.embeddingMatrix 
dot_product_matrix = np.dot(matrixA.T, matrixA)

mean_squared_difference = imgcalc.get_MSE(matrixG, dot_product_matrix)
print("Mean Squared Difference of Pencorr (A'A) and NCC (G):", mean_squared_difference)

model_vectors = imgcalc.get_vector_embeddings(input_dataset, model)
model_matrix = imgcalc.get_matrix_embeddings(input_dataset, model_vectors)
        
mean_squared_difference = imgcalc.get_MSE(matrixG, model_matrix.detach().cpu().numpy())
print("\nMean Squared Difference of Model and NCC (G):", mean_squared_difference)
        
mean_squared_difference = imgcalc.get_MSE(dot_product_matrix, model_matrix.detach().cpu().numpy())
print("Mean Squared Difference of Model and Pencorr (A'A):", mean_squared_difference)
    
print(f"\nPrinting matrices")
print(f"\nMatrix G: {matrixG}")
print(f"\nMatrix A'A (Pencorr): {dot_product_matrix}, {dot_product_matrix.shape}")
print(f"\nMatrix A'A (Model): {model_matrix.detach().cpu().numpy()}, {model_matrix.detach().cpu().numpy().shape}")

print(f"\nPrinting vectors")
print(f"\nEmbedding of image {index1} for Pencorr (A'A): {matrixA[:,index1]}")
print(f"Embedding of image {index1} for Model: {model_vectors[index1]}")

print(f"\nDifferences between vector embeddings")
mean_squared_difference = imgcalc.get_MSE(matrixA[:,index1], model_vectors[index1].detach().cpu().numpy())
print(f"Mean Squared Difference of Pencorr (A) and Model: {mean_squared_difference}")

In [ ]:
#-----------------After transforming output embedding matrices via Orthorgonal Procrustes ----------------
print(f"Difference between matrices")
model_vectors = imgcalc.get_vector_embeddings(input_dataset, model)
model_transformed, error_model = imgcalc.get_orthogonal_transformation(model_vectors, matrixA) #transposed
model_transformed = [torch.tensor(row, dtype=torch.float32, device= device).unsqueeze(0).requires_grad_() for row in model_transformed]
model_matrix_transformed = imgcalc.get_matrix_embeddings(input_dataset, model_transformed)

mean_squared_difference = imgcalc.get_MSE(matrixG, model_matrix_transformed.detach().cpu().numpy())
print("\nMean Squared Difference of Model and NCC (G):", mean_squared_difference)
        
mean_squared_difference = imgcalc.get_MSE(dot_product_matrix, model_matrix_transformed.detach().cpu().numpy())
print("Mean Squared Difference of Model and Pencorr (A'A):", mean_squared_difference)
     

print(f"\nPrinting matrices after transformation")
print(f"\nMatrix G: {matrixG}")
print(f"\nMatrix A'A (Pencorr): {dot_product_matrix}, {dot_product_matrix.shape}")
print(f"\nMatrix A'A (Model): {model_matrix_transformed}, error: {error_model}")

print(f"\nPrinting vectors ")
print(f"\nEmbedding of image {index1} for Pencorr (A'A): {matrixA[:,index1]}")
print(f"Embedding of image {index1} for Model: {model_transformed[index1]}")

print(f"\nDifferences between vector embeddings")
mean_squared_difference = imgcalc.get_MSE(matrixA[:,index1], model_transformed[index1].detach().cpu().numpy())
print(f"Mean Squared Difference of Pencorr (A) and Transformed Model: {mean_squared_difference}")

# magnitude = np.linalg.norm(model_transformed[index1].detach().cpu().numpy())
# print("magnitude: ", magnitude)
# vector = model_transformed[index1].detach().cpu().numpy()/magnitude
# print("vector after normalisation: ", vector)

# mean_squared_difference = imgcalc.get_MSE(matrixA[:,index1], vector)
# print(f"Mean Squared Difference of Pencorr (A) and Normalised TransformedModel: {mean_squared_difference}")
# magnitude = np.linalg.norm(vector)
# print("magnitude: ", magnitude)

In [ ]:
# ------------------------ MSE of embedding across dataset -----------------------------
num = len(input_dataset)
MSE_transformed = []
MSE_original = []
print(f"\nMSE between vector embeddings of Pencorr (A) and Model:")   
for i in range(num):
    difference_squared = (matrixA[:,i] - model_transformed[i].detach().cpu().numpy()) ** 2
    mean_squared_difference = np.sum(difference_squared) / difference_squared.size
    #print(f"Transformed matrix of Index {i}: {mean_squared_difference}")
    MSE_transformed.append(mean_squared_difference)
    difference_squared = (matrixA[:,i] - model_vectors[i].detach().cpu().numpy()) ** 2
    mean_squared_difference = np.sum(difference_squared) / difference_squared.size
    #print(f"Original matrix of Index {i}: {mean_squared_difference}")
    MSE_original.append(mean_squared_difference)
    
imgplt.plot_score_distribution(MSE_transformed, "MSE of vector embeddings for Transformed Model")
imgplt.plot_score_distribution(MSE_original, "MSE of vector embeddings for Model")

In [ ]:
# ------------------------ MSE of embedding across dimensions across dataset -----------------------------
dimensions = [32, 64, 128, 192, 256, 384, 512]
for dim in dimensions:
    
    embeddingType = f"pencorr_{dim}"
    sampleName = f"{imageType} {filters} {percentage} sample"

    sampleEstimator = SampleEstimator(sampleName=sampleName, embeddingType=embeddingType, imageProductType=imageProductType)
    input_dataset = []
    for img in testSample:
        img_tensor = torch.from_numpy(np.array(img, dtype=np.float64))
        img_tensor = img_tensor.unsqueeze(0).unsqueeze(0).to(device).double()
        input_dataset.append(img_tensor)
    input_dataset = [tensor.float() for tensor in input_dataset] 
    stacked_tensor = torch.stack(input_dataset)
    input_dataset = stacked_tensor.cpu().numpy()      
    input_dataset = [torch.tensor(data).to(device).float() for data in input_dataset]
    num = len(input_dataset)
    
    model = models.SimpleCNN(dimensions=dim, padding_mode='circular').to(device)
    model.load_state_dict(torch.load(os.path.join(os.path.abspath("../../VecRepV3"), "model", 
                     f"best_model_batch_greyscale_8bin_LPS_circular_{dim}d.pt"), weights_only=True))

    model.eval()
    
    matrixG = sampleEstimator.imageProductMatrix  
    matrixA = sampleEstimator.embeddingMatrix 
    dot_product_matrix = np.dot(matrixA.T, matrixA)
    
    model_vectors = imgcalc.get_vector_embeddings(input_dataset, model)
    model_transformed, error_model = imgcalc.get_orthogonal_transformation(model_vectors, matrixA) #transposed
    model_transformed = [torch.tensor(row, dtype=torch.float32, device= device).unsqueeze(0).requires_grad_() for row in model_transformed]
    
    MSE_transformed = []
    MSE_original = []
    print(f"\nMSE between vector embeddings of dimension {dim} for Pencorr (A) and Model:")   
    for i in range(num):
        difference_squared = (matrixA[:,i] - model_transformed[i].detach().cpu().numpy()) ** 2
        mean_squared_difference = np.sum(difference_squared) / difference_squared.size
        #print(f"Transformed matrix of Index {i}: {mean_squared_difference}")
        MSE_transformed.append(mean_squared_difference)
        difference_squared = (matrixA[:,i] - model_vectors[i].detach().cpu().numpy()) ** 2
        mean_squared_difference = np.sum(difference_squared) / difference_squared.size
        #print(f"Original matrix of Index {i}: {mean_squared_difference}")
        MSE_original.append(mean_squared_difference)
        
    imgplt.plot_score_distribution(MSE_transformed, f"MSE of Transformed Model")
    imgplt.plot_score_distribution(MSE_original, f"MSE of Original Model")